In [31]:
import pandas as pd
from pathlib import Path
import numpy as np
import nltk

In [62]:
signoffs = ["Best",
"All the best",
"Best wishes",
"Best regards",
"Sincerely",
"Respectfully",
"Regards",
"Warm regards",
"Kind regards",
"Thank you,",
"Thank you in advance,",
"Talk to you soon",
"Thanks,"]

In [110]:
all_data = []
for d in Path("/path/to/enron/").iterdir():
    name = d.as_posix().split("/")[-1]
    for file in Path(d / "sent_items").rglob("*."):
        with open(file, 'rb') as f:
            try:
                text = f.read().decode().strip()
                text = text.split("-----Original Message-----")[0]
                
                if in text or "Reuters" in text:
                    continue
                text = " ".join(text.split("\r")[-1].strip().split())
                text = text.split("From:")[0]
                
                text = text.replace("\\'", "'")
                
                signed = False
                for s in signoffs:
                    if s in text:
                        text = " ".join(text.split(s)[:-1]).strip()
                        signed = True
                        break
                if signed == False and text.endswith(".") == False and text.endswith("?") == False and text.endswith("!") == False:
                    text = " ".join(text.split()[:-1])
                
                if text == " " or text == "":
                    text = None
            except UnicodeDecodeError:
                continue
            all_data.append({"author":name, "text":text})
len(all_data)

36966

In [111]:
df = pd.DataFrame(all_data).dropna().reset_index().drop("index", axis=1)
df

,author,text
0,lokey-t,Both are good for me.
1,lokey-t,Got it.
2,lokey-t,If Chattahoochee converted all of their SFTS t...
3,lokey-t,Kevin - We estimate the rate base at the end o...
4,lokey-t,Today's Inside FERC has a story indicating tha...
...,...,...
31029,king-j,"Hey mom, Thanks for the info on Stan. What are..."
31030,king-j,That was great.
31031,king-j,me too......gotta clean the
31032,king-j,"Hey buddy, I'm closing on my new house that we..."


In [112]:
df.iloc[-2].text

"Hey buddy, I'm closing on my new house that week so that may be tough. There is a chance that i could get away on Saturday morning so let me know your schedule. Hope the family is well."

In [113]:
df.to_csv("enron_sent_all.csv", index=False)

In [114]:
df.author.value_counts()

dasovich-j    1287
steffes-j     1236
germany-c     1178
kaminski-v     977
kitchen-l      953
              ... 
badeer-r         5
quenet-j         5
gay-r            4
phanis-s         3
shapiro-r        1
Name: author, Length: 136, dtype: int64

In [115]:
p = np.percentile(list(dict(df.author.value_counts()).values()), 80)
p

388.0

In [116]:
top = [k for k, v in dict(df.author.value_counts()).items() if v >= int(p)]
len(top)

28

In [117]:
topdf = df[df.author.isin(top)]
topdf

,author,text
331,dasovich-j,His name is Sean Steele. I'll have his number ...
332,dasovich-j,Don't hate me for sending a note. What do you ...
333,dasovich-j,Here's a cut at the taxonomy of regulatory ris...
334,dasovich-j,Thanks. I'm in Goucester Mass this week but am...
335,dasovich-j,Can I distribute to the business groups?
...,...,...
30883,blair-l,John please send this to Shelly Thanks Lynn --...
30884,blair-l,FYI. Thanks.
30885,blair-l,"Ranelle, I have a meeting Tuesday afternoon fr..."
30886,blair-l,FYI. Thought you would be interested since mos...


In [118]:
topdf.iloc[123].text

"Am I reading this right? The gas contracts are $5B out of the money (from CA's perspective)? Are they straight gas deals?"

In [119]:
topdf.author.value_counts()

dasovich-j       1287
steffes-j        1236
germany-c        1178
kaminski-v        977
kitchen-l         953
watson-k          897
presto-k          862
blair-l           814
heard-m           715
lenhart-m         684
ward-k            601
arnold-j          601
scott-s           565
corman-s          561
lavorato-j        551
nemec-g           540
shackleton-s      526
perlingiere-d     514
dorland-c         489
love-p            486
cash-m            481
hayslett-r        476
williams-w3       459
quigley-d         444
taylor-m          435
parks-j           427
white-s           400
forney-j          388
Name: author, dtype: int64

In [120]:
topdf.to_csv("enron.csv", index=False)

# filter

In [121]:
df = pd.read_csv("enron.csv")

In [129]:
mask = [True if len(nltk.sent_tokenize(x)) > 1 else False for x in df.text]

In [136]:
filtered = df[mask]
filtered.iloc[2].text

"Who loves ya baby? Hey, don't say anything to Karen, or her parents, but I'd like to get something for her parents as a thank you for Gloucester. What sorts of things do they like? You heard that Prentice got a job offer from Price Waterhouse?! See you on Friday!"

In [132]:
filtered.to_csv("enron.csv", index=False)